In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 75ece029-550e-4a44-af96-f44a31386e5f, 5, Finished, Available, Finished)

In [4]:
%%sql
create table if not EXISTS Project_Lakehouse.gold_fact_sale
(
Order_ID	string	,
Price float,
Quantity	float	,
Sales	float	,
Discount	float	,
Profit	float	,
Shipping_Cost	float	,
Order_Date	date	,
Shipping_Date	date	,
Product_ID	long	,
OrderPriority_ID	Long	,
ShipMode_ID	Long	,
Customer_ID	string	,
Order_Year	integer	,
Order_Month	integer	,
Created_TS	timestamp	,
Modified_TS	timestamp	
)
using DELTA
PARTITIONED by (Order_Year,Order_Month)

StatementMeta(, 75ece029-550e-4a44-af96-f44a31386e5f, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
Max_Date=spark.sql("select coalesce(max(Modified_TS),'1900-01-01') from Project_Lakehouse.Gold_Fact_Sale").first()[0]

StatementMeta(, 75ece029-550e-4a44-af96-f44a31386e5f, 7, Finished, Available, Finished)

In [6]:
Max_Date

StatementMeta(, 75ece029-550e-4a44-af96-f44a31386e5f, 8, Finished, Available, Finished)

'1900-01-01'

In [8]:
df_bronze= spark.sql("""select 
bronze_sales.Order_ID	,
bronze_sales.Sales as Price,
bronze_sales.Quantity,
bronze_sales.Sales	* bronze_sales.Quantity as Sales,
bronze_sales.Discount	,
bronze_sales.Profit	,
bronze_sales.Shipping_Cost	,
bronze_sales.Order_Date	,
bronze_sales.Shipping_Date	,
gold_product.Product_ID	,
gold_orderpriority.OrderPriority_ID	,
gold_shipmode.ShipMode_ID	,
bronze_sales.Customer_ID	,
Year(Order_Date) as Order_Year	,
Month(Order_Date) as Order_Month	
from Project_Lakehouse.bronze_sales 
inner join Project_Lakehouse.gold_product on bronze_sales.Product=gold_product.Product and 
                                             bronze_sales.Product_Category=gold_product.Product_Category
inner join Project_Lakehouse.gold_shipmode on bronze_sales.Ship_Mode= gold_shipmode.Ship_Mode
inner join Project_Lakehouse.gold_orderpriority on bronze_sales.Order_Priority=gold_orderpriority.Order_Priority
""")

StatementMeta(, 75ece029-550e-4a44-af96-f44a31386e5f, 10, Finished, Available, Finished)

In [9]:
df_bronze.createOrReplaceTempView("ViewFactSale")

StatementMeta(, 75ece029-550e-4a44-af96-f44a31386e5f, 11, Finished, Available, Finished)

In [10]:
%%sql
merge into Project_Lakehouse.gold_fact_sale as gfs
using ViewFactSale as vfs 
on gfs.Order_Year=vfs.Order_Year and gfs.Order_Month=vfs.Order_Month and gfs.Order_ID=vfs.Order_ID
when matched then 
update SET
gfs.Sales	=	vfs.Sales	,
gfs.Price	=	vfs.Price	,
gfs.Quantity	=	vfs.Quantity	,
gfs.Discount	=	vfs.Discount	,
gfs.Profit	=	vfs.Profit	,
gfs.Shipping_Cost	=	vfs.Shipping_Cost	,
gfs.Order_Date	=	vfs.Order_Date	,
gfs.Shipping_Date	=	vfs.Shipping_Date	,
gfs.Product_ID	=	vfs.Product_ID	,
gfs.OrderPriority_ID	=	vfs.OrderPriority_ID	,
gfs.ShipMode_ID	=	vfs.ShipMode_ID	,
gfs.Customer_ID	=	vfs.Customer_ID	,
gfs.Modified_TS	=	current_timestamp()	

when not matched then
INSERT
(
gfs.Order_ID	,
gfs.Sales	,
gfs.Price,
gfs.Quantity	,
gfs.Discount	,
gfs.Profit	,
gfs.Shipping_Cost	,
gfs.Order_Date	,
gfs.Shipping_Date	,
gfs.Product_ID	,
gfs.OrderPriority_ID	,
gfs.ShipMode_ID	,
gfs.Customer_ID	,
gfs.Order_Year	,
gfs.Order_Month	,
gfs.Created_TS	,
gfs.Modified_TS	

)
VALUES
(
vfs.Order_ID	,
vfs.Sales	,
vfs.Price,
vfs.Quantity	,
vfs.Discount	,
vfs.Profit	,
vfs.Shipping_Cost	,
vfs.Order_Date	,
vfs.Shipping_Date	,
vfs.Product_ID	,
vfs.OrderPriority_ID	,
vfs.ShipMode_ID	,
vfs.Customer_ID	,
vfs.Order_Year	,
vfs.Order_Month	,
current_timestamp()	,
current_timestamp()

)

StatementMeta(, 75ece029-550e-4a44-af96-f44a31386e5f, 12, Finished, Available, Finished)

<Spark SQL result set with 1 rows and 4 fields>